# IBM Data Science Professional Certificate Capstone

This notebook is for the neighborhood analyzation project for the data science capstone course on Coursera.

## Introduction
Gainesville, FL is Florida's 15th largest city with 134,945 people within the city and 295,266 in the overall area and it has grown 8% between 2010 and 2020. As it continues to grow, we know there will be opportunities for new restaraunts including new pizza places.

To identify the best potential areas for our new pizza place, we will analyze the different parts of the city to identify what areas would have the least competition for a pizza place. However, a challenge with Gainesville is that the population is spread out into several dense areas and many less dense areas.

As such, we will want to identify areas that have low competition for a pizza place but also have enough people in the area to support a new establishment.

In [1]:
# Our needed imports.
#!conda install -c conda-forge folium --yes
#!conda install -c conda-forge geopy --yes
import folium
import ibm_boto3
import json
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests
import types
from botocore.client import Config
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from IPython.display import Image 
from sklearn.cluster import KMeans

In [14]:
# Create our corners of Gainesville.
gainesville_north = 29.711381
gainesville_south = 29.596737
gainesville_west = -82.453961
gainesville_east = -82.262119

In [40]:
# Define how many rows and columns we want to create for districts.
DISTRICT_ROWS = 10
DISTRICT_COLUMNS = 15
GAINESVILLE_LATITUDE = 29.662737
GAINESVILLE_LONGITUDE = -82.370212

In [41]:
# Calculate how big each segment is.
lat_diff = gainesville_north - gainesville_south
long_diff = gainesville_west - gainesville_east
lat_segment = lat_diff / (DISTRICT_ROWS)
long_segment = long_diff / (DISTRICT_COLUMNS)

In [42]:
# Generate the center for all segments.
gainesville_districts = list()
north_boundary = gainesville_north
for row in range(DISTRICT_ROWS):
    south_boundary = north_boundary - lat_segment
    row_center = (north_boundary + south_boundary) / 2
    west_boundary = gainesville_west
    for column in range(DISTRICT_COLUMNS):
        east_boundary = west_boundary - long_segment
        column_center = (east_boundary + west_boundary) / 2
        west_boundary = east_boundary
        gainesville_districts.append((row_center, column_center))
    north_boundary = south_boundary
gainesville_districts[0:2]

[(29.7056488, -82.44756626666668), (29.7056488, -82.43477680000001)]

In [43]:
# Create map of Gainesville to see our districts.
general_map = folium.Map(location=[GAINESVILLE_LATITUDE, GAINESVILLE_LONGITUDE], zoom_start=12)

# Add markers to the map for each districts.
for lat, lng in gainesville_districts:
    folium.CircleMarker(
        [lat, lng],
        radius=17,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(general_map)  
    
general_map